In [7]:
import yaml
import json

def yaml_to_tree_positions(yaml_data):
    reactflow = {"nodes": [], "edges": []}
    nodes = yaml_data.get("nodes", [])
    edges = yaml_data.get("edges", [])

    # Create a mapping of edges to track parent-child relationships
    children_map = {}
    for edge in edges:
        parent = edge["from"]
        child = edge["to"]
        if parent not in children_map:
            children_map[parent] = []
        children_map[parent].append(child)

    # Helper function to assign positions recursively
    def assign_positions(node_id, depth, x_offset, x_step):
        if node_id not in node_positions:
            node_positions[node_id] = {"x": x_offset, "y": depth * y_spacing}

            # Add node to ReactFlow nodes
            node = next((n for n in nodes if n["name"] == node_id), None)
            if node:
                reactflow["nodes"].append({
                    "id": node_id,
                    "data": {
                        "function": node.get("function", ""),
                        "reducer": node.get("reducer", ""),
                        "label": node_id.replace("_", " ").title()
                    },
                    "position": {"x": x_offset, "y": depth * y_spacing}
                })

            # Recurse to children
            if node_id in children_map:
                num_children = len(children_map[node_id])
                child_x_offset = x_offset - (x_step * (num_children - 1)) / 2
                for index, child_id in enumerate(children_map[node_id]):
                    assign_positions(
                        child_id,
                        depth + 1,
                        child_x_offset + (index * x_step),
                        x_step // 2
                    )

    # Root node position configuration
    y_spacing = 150  # Vertical distance between levels
    x_step = 400  # Horizontal distance between child nodes
    node_positions = {}

    # Assume START is the root node
    assign_positions("START", 0, 0, x_step)

    # Add edges to ReactFlow
    for edge in edges:
        reactflow["edges"].append({
            "id": f'{edge["from"]}_to_{edge["to"]}',
            "source": edge["from"],
            "target": edge["to"]
        })

    return reactflow

# Example YAML input
yaml_content = """
stategraph:
  nodes:
    - name: diet_search
      function: diet_search_node
      reducer: default
    - name: diet_logger
      function: diet_logging_node
      reducer: default
    - name: exercise_search
      function: exercise_search_node
      reducer: default
    - name: exercise_logger
      function: exercise_logging_node
      reducer: default
    - name: diet_supervisor
      function: diet_supervisor_node
      reducer: default
    - name: exercise_supervisor
      function: exercise_supervisor_node
      reducer: default
    - name: teams_supervisor
      function: teams_supervisor_node
      reducer: default
    - name: call_diet_team
      function: call_diet_team
      reducer: default
    - name: call_exercise_team
      function: call_exercise_team
      reducer: default
  edges:
    - from: START
      to: teams_supervisor
    - from: teams_supervisor
      to: diet_supervisor
    - from: teams_supervisor
      to: exercise_supervisor
    - from: diet_supervisor
      to: diet_search
    - from: diet_supervisor
      to: diet_logger
    - from: exercise_supervisor
      to: exercise_search
    - from: exercise_supervisor
      to: exercise_logger
    - from: diet_search
      to: teams_supervisor
    - from: diet_logger
      to: teams_supervisor
    - from: exercise_search
      to: teams_supervisor
    - from: exercise_logger
      to: teams_supervisor
"""

# Parse YAML
parsed_yaml = yaml.safe_load(yaml_content)

# Convert to ReactFlow JSON with tree layout
reactflow_json = yaml_to_tree_positions(parsed_yaml["stategraph"])

# Output as JSON
print(json.dumps(reactflow_json, indent=2))


{
  "nodes": [
    {
      "id": "teams_supervisor",
      "data": {
        "function": "teams_supervisor_node",
        "reducer": "default",
        "label": "Teams Supervisor"
      },
      "position": {
        "x": 0.0,
        "y": 150
      }
    },
    {
      "id": "diet_supervisor",
      "data": {
        "function": "diet_supervisor_node",
        "reducer": "default",
        "label": "Diet Supervisor"
      },
      "position": {
        "x": -100.0,
        "y": 300
      }
    },
    {
      "id": "diet_search",
      "data": {
        "function": "diet_search_node",
        "reducer": "default",
        "label": "Diet Search"
      },
      "position": {
        "x": -150.0,
        "y": 450
      }
    },
    {
      "id": "diet_logger",
      "data": {
        "function": "diet_logging_node",
        "reducer": "default",
        "label": "Diet Logger"
      },
      "position": {
        "x": -50.0,
        "y": 450
      }
    },
    {
      "id": "exercise_super